In [ ]:
import json 
import pandas as pd
import numpy as np

from sklearn.preprocessing import OneHotEncoder

from tqdm import tqdm

trainfile = "train.jsonl"
batch_size = 5000
start = 0

def loadBatchData(start, batch_size, filename):
    trainset = []
    with open(filename, "r", encoding="utf-8") as file:
        for id, line in enumerate(file):
            if id < start:
                continue
            if id >= start + batch_size:
                break
            try:
                data = json.loads(line)  # Convert JSON string to dictionary
                trainset.append(data)
            except: 
                print("bad lines")
                continue
            
    return trainset
            
trainset = loadBatchData(start, batch_size, trainfile)
        

In [2]:
# define action space
def getItems(trainset) -> set:
    actions = [] # items
    for session in trainset:
        for i in [i["aid"] for i in session["events"]]:
            actions.append(i)
    return set(actions)


In [3]:
def getEncoding(items: set) -> dict:
    """
    Output:
        key: item id
        data: one hot encoding
    """
    
    data = np.array([list(items)])
    encoder = OneHotEncoder(sparse_output=False)
    one_hot = encoder.fit_transform(data)
    
    encoding = {}
    for i in encoder.get_feature_names_out():
        encode = i.split("_")
        data = int(encode[0][1:])
        key = int(encode[1])
        encoding[key] = data  

    return encoding


items = getItems(trainset)
encoding = getEncoding(items)
print(len(items))

125707


In [ ]:
def getEmbeddingTrainSet(trainset, context_window: int = 60*60*24*1000) -> tuple[list, list]:
    """
    X[0]: [0 ... 0 1 0 ... 0] index at X[0] = 1
    y[0]: [0...0 1 0...0 ] index at y[0] = 1
    y[0] for compute loss
    """
    trainset[0]
    context_window = 60*60*24*1000 # 1 day
    X = [] # Input
    y = [] # Near
    for session in tqdm(trainset):
        session_ = pd.DataFrame(session["events"])
        for ev in session["events"]:
            X.append(ev["aid"])
            y.append(list(session_[(session_["ts"] < ev["ts"] + context_window) & (session_["ts"] > ev["ts"] - context_window)]["aid"]))
    return X, y

In [9]:

"""
X[0]: [0 ... 0 1 0 ... 0] index at X[0] = 1
y[0]: [0...0 1 0...0 ] index at y[0] = 1
y[0] for compute loss
"""
context_window = 60*60*24*1000 # 1 day
X = [] # Input
y = [] # Near
for session in tqdm(trainset):
    session_ = pd.DataFrame(session["events"])
    for ev in session["events"]:
        X.append(ev["aid"])
        y.append(list(session_[(session_["ts"] < ev["ts"] + context_window) & (session_["ts"] > ev["ts"] - context_window)]["aid"]))
        

100%|██████████| 5000/5000 [02:46<00:00, 30.10it/s]


In [10]:
import torch

print("Torch Version:", torch.__version__)  # Print PyTorch version
print("CUDA Available:", torch.cuda.is_available())  # Check if GPU is available

if torch.cuda.is_available():
    print("GPU Name:", torch.cuda.get_device_name(0))  # Print GPU name


ModuleNotFoundError: No module named 'torch'

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

# Define a Two-Layer Neural Network
class TwoLayerNet(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(TwoLayerNet, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)  # First layer
        self.relu = nn.ReLU()  # Activation function
        self.fc2 = nn.Linear(hidden_size, output_size)  # Output layer

    def forward(self, x):
        x = self.fc1(x)  # First layer
        x = self.relu(x)  # Activation function
        x = self.fc2(x)  # Output layer
        return x

# Hyperparameters
input_size = 10   # Example input size (features)
hidden_size = 32  # Hidden layer size
output_size = 2   # Example output size (binary classification)

# Initialize model, loss, and optimizer
model = TwoLayerNet(input_size, hidden_size, output_size)
criterion = nn.CrossEntropyLoss()  # Suitable for classification tasks
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Example Training Loop (with dummy data)
for epoch in range(10):  # 10 epochs
    inputs = torch.rand(5, input_size)  # 5 sample inputs
    labels = torch.randint(0, output_size, (5,))  # Random target labels

    optimizer.zero_grad()  # Reset gradients
    outputs = model(inputs)  # Forward pass
    loss = criterion(outputs, labels)  # Compute loss
    loss.backward()  # Backpropagation
    optimizer.step()  # Update weights

    print(f"Epoch [{epoch+1}/10], Loss: {loss.item():.4f}")

# Example Prediction
test_input = torch.rand(1, input_size)  # Single test sample
predicted = model(test_input)
print("\nPredicted Output:", predicted)


In [ ]:


# Given timestamp in milliseconds
timestamp_ms = trainset[0]["events"][0]["ts"]

# Convert milliseconds to seconds
timestamp_sec = timestamp_ms / 1000

# Create a datetime object from the timestamp
dt = datetime.datetime.fromtimestamp(timestamp_sec)

print("Datetime:", dt)


Datetime: 2022-08-01 06:00:00.025000


In [ ]:
for k, d in encoding:
    print(d)

TypeError: cannot unpack non-iterable int object